<a href="https://colab.research.google.com/github/JuanD4vy/Low_cost_GPT/blob/main/DL3/NLPCODE_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#***Universidad Militar Nueva Granada***
**Programa de ingeniería en Telecomunicaciones**

**Elaborado por: César González Santoyo y Juan David Aviles Palma**

Dataset ag_news: https://huggingface.co/datasets/fancyzhx/ag_news/viewer/default/train?f[label][value]=0

# Step 0: Instalar dependencias

In [1]:
!pip install transformers[torch] accelerate -U

In [2]:
!pip install accelerate -U

In [3]:
!pip install accelerate

In [4]:
# Instalación de las dependencias necesarias
!pip install transformers datasets accelerate -U

#Step 1: Dataset Loading and Preprocessing

In [5]:
import time
from datasets import load_dataset
from transformers import AutoTokenizer

# Cargar una fracción del dataset AG News
def load_partial_dataset():
    dataset = load_dataset('ag_news')
    small_train_dataset = dataset['train'].shuffle(seed=42).select(range(1000))  # Utilizar solo 1000 ejemplos
    small_test_dataset = dataset['test'].shuffle(seed=42).select(range(200))     # Utilizar solo 200 ejemplos
    return small_train_dataset, small_test_dataset

train_dataset, test_dataset = load_partial_dataset()

# Utilizar el tokenizer de DistilBERT
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)

# Tokenizar los datos
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

#Step 2: Define the Model and Training Arguments

In [6]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

# Cargar el modelo DistilBERT para clasificación de secuencias
model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=4)

# Definir los argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=16,  # Aumentar tamaño del batch si es posible
    per_device_eval_batch_size=16,
    num_train_epochs=1,  # Reducir a 1 época
    save_steps=10_000,
    save_total_limit=2,
    fp16=False,  # Deshabilitar precisión mixta
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


#Step 3: Train the Model

In [7]:
# Crear el trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Medir el tiempo de entrenamiento
start_time = time.time()

# Entrenar el modelo
trainer.train()

end_time = time.time()
training_time = end_time - start_time
print(f"Training completed in {training_time / 60:.2f} minutes.")

Epoch,Training Loss,Validation Loss
1,No log,0.882599


Training completed in 62.83 minutes.


#Step 4: Save the Model

In [8]:
# Guardar el modelo entrenado
model.save_pretrained('./trained_model')
tokenizer.save_pretrained('./trained_model')

('./trained_model/tokenizer_config.json',
 './trained_model/special_tokens_map.json',
 './trained_model/vocab.txt',
 './trained_model/added_tokens.json',
 './trained_model/tokenizer.json')

#Step 5: Make Inferences

In [15]:
from transformers import pipeline

# Cargar el modelo y el tokenizer
model = AutoModelForSequenceClassification.from_pretrained('./trained_model')
tokenizer = AutoTokenizer.from_pretrained('./trained_model')

# Crear el pipeline de clasificación de texto
classifier = pipeline('text-classification', model=model, tokenizer=tokenizer)

# Realizar una clasificación
text = "Nadal Wins in Poland for First ATP Title (AP) AP - Spain's Rafael Nadal won his first ATP singles title Sunday, beating Argentina's Jose Acasuso 6-3, 6-4 in the final at the Idea Prokom Open."
result = classifier(text)

print(result)

[{'label': 'LABEL_1', 'score': 0.5157055258750916}]
